In [1]:
import os
import sqlite3
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm

In [2]:
BG_DIR = "../data/background"

In [6]:
def load_grid(filename: str = "RSS_Skane_squares.gpkg") -> gpd.GeoDataFrame:
    """
    Load the grid data from a GPKG file.

    Args:
        filename (str): The name of the GPKG file containing the grid data. Defaults to "RSS_Skane_squares.gpkg".

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the grid data with columns 'rid' and 'geometry'.

    Example:
        >>> grid = load_grid()
        >>> grid.columns
        Index(['rid', 'geometry'], dtype='object')
    """
    path = os.path.join(BG_DIR, filename)

    if not os.path.isfile(path):
        raise FileNotFoundError(f"Grid file not found: {path}")

    cols = ["rut_id", "geometry"]
    crs = "EPSG:3006"

    gdf = load_gpkg(path, cols, crs)

    # Format the columns
    gdf.rename(columns={"rut_id": "rid"}, inplace=True)
    gdf = gdf.astype({"rid": np.uint64})

    gdf = gdf.dissolve(by="rid")
    gdf = gdf.reset_index()

    tqdm.write(f"Loaded GeoDataFrame grid ({filename}).")
    return gdf

def set_crs(gdf: gpd.GeoDataFrame, crs: str = None) -> gpd.GeoDataFrame:
    """Set the coordinate reference system (CRS) for a GeoDataFrame."""
    if crs:
        if gdf.crs is None:
            gdf.set_crs(crs, allow_override=True, inplace=True)
        if gdf.crs != crs:
            gdf.to_crs(crs, inplace=True)

    return gdf

def load_gpkg(path: str, cols: list[str] = None, crs: str = None) -> gpd.GeoDataFrame:
    """Load the GPKG file."""
    gdf = gpd.read_file(path)
    gdf = set_crs(gdf, crs)

    if cols:
        return gdf[cols]
    return gdf



In [7]:


region = "10"
grid = load_grid()
# kommuner = load_kommun()
# natomrade = load_natomrade()

# path = db_path(region)
# conn, cursor = db_connect(path)
# tables = db_tables(cursor)
# years = db_years(tables)

DataSourceError: sqlite3_prepare_v2(SELECT COUNT(*) FROM sqlite_master WHERE name IN ('gpkg_metadata', 'gpkg_metadata_reference') AND type IN ('table', 'view')) failed: database is locked